In [5]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [ ]:
epochs = 100
batch_size = 64
patience = 10
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [ ]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(30, 30),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.Conv2D(32, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(32, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(64,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(64,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(6,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

In [ ]:
train = utils.image_dataset_from_directory(
    "images/Fingers", # Change path
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "images/Fingers", # Change path
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

In [ ]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

In [ ]:
# Salvar imagens
def SaveImage(path, file, img):
    if not os.path.exists(path):
        os.makedirs(path)
    cv.imwrite(f'{path}/{file}', img)
    
# Transformada de Fourier
def fft(img):
    img = np.fft.fft2(img)
    img = np.fft.fftshift(img)
    return img

# Inversa (retorna para imagem original)
def ifft(fimg):
    fimg = np.fft.ifftshift(fimg)
    fimg = np.fft.ifft2(fimg)
    return fimg

# Obtém a magnitude da imagem
def mag(img):
    absvalue = np.abs(img)
    magnitude = 20 * np.log(absvalue)
    return magnitude

# Normaliza a imagem entre 0 e 255
def norm(img):
    img = cv.normalize(
    img, None, 0, 255,
    cv.NORM_MINMAX
    )

# Melhor para ver imagens da transformada e imagens pequenas em geral.
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    return img

def Fourier(path):
    img = cv.imread(path)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    img = fft(img)
    img = mag(img)
    norm(img)
    
    return img

In [ ]:
# Aplica Fourier nas Imagens

path = 'images/Fingers' # Change path
save_path = 'images/Fourier' # Change path

for dir in os.listdir(path):
    dir_path = f'{path}/{dir}'
    for file in os.listdir(dir_path):
        img = Fourier(f'{dir_path}/{file}')
        SaveImage(f'{save_path}/{dir}', file, img)

In [ ]:
path = 'images/Fingers' # Change path
save_path = 'images/Fourier' # Change path

for dir in os.listdir(path):
    dir_path = f'{path}/{dir}'
    for file in os.listdir(dir_path):
        # algoritmo de segmentação
        
        # dar crop na imagem
        
        img = cv.dilate(img, np.ones((2, 2)))
        img = cv.erode(img, np.ones((10, 10)))
        SaveImage(f'{save_path}/{dir}', file, img)



In [20]:
def find(img, x, y):
    x0 = x
    xf = x
    y0 = y
    yf = y
    
    queue = [
        (x + 1, y),
        (x - 1, y),
        (x,y + 1),
        (x, y - 1)
    ]
    
    while len(queue) > 0:
        x, y = queue.pop()
        
        if y < 0 or y >= len(img):
            continue
        row = img[y]
        
        if x < 0 or x >= len(img):
            continue
        pixel = row[x]
        
        if pixel == 255:
            continue
        img[y][x] = 255
        
        x0 = min(x0, x)
        xf = max(xf, x)
        y0 = min(y0, y)
        yf = min(yf, y)
        
        queue.append((x + 1, y))
        queue.append((x - 1, y))
        queue.append((x, y + 1))
        queue.append((x, y - 1))
        
    cropped_img = img[y0 : yf + 1, x0 : xf + 1 ]
    cropped_img = cv.resize(cropped_img, (128, 128))
    
    return (cropped_img)
        

In [21]:
if __name__ == "__main__":
    image = cv.imread('teste.png', cv.IMREAD_GRAYSCALE)
    
    x = 20
    y = 20
    
    cropped_image = find(image, x, y)
    
    cv.imshow('Cropped Image', cropped_image)
    cv.waitKey(0)
    cv.destroyAllWindows()